In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

data = pd.read_csv('./data/review_before.csv')
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.columns = ['제품명', '닉네임', '나이', '피부타입', '성별', '평점','리뷰내용']
data

# 평점 기준
# 긍정 : 5, 4
# 중립 : 3
# 부정 : 2, 1

data['평점']

def rating_replace(rating) :
  if rating == 5 or rating == 4 : # 긍정은 2로 변경
    return 2
  elif rating == 3 : # 중립은 1으로 변경
    return 1
  else : # 부정은 0로 변경
    return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 전처리
X = data['리뷰내용']
y = data['감성분석 결과']

# 라이브러리 불러오기
from konlpy.tag import Okt
okt = Okt()

okt.tagset

list_ = []

for i in X:
  list_.append(okt.pos(i, join=True))

list_


tagging = []
morph = []
tag = []

for j in range(len(list_)):
  for i in list_[j] :
    morph.append(i.split('/')[0])
    tag.append(i.split('/')[1])
  tagging.append([morph, tag])

pd.DataFrame(tagging, columns =['morph', 'tag'])

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('Verb' in d.index) | ('Noun' in d.index) | ('Adjective' in d.index):
        return d.loc[d.index.intersection(['Verb','Noun','Adjective']),'morph'].values
    else :
        return []

from sklearn.feature_extraction.text import TfidfVectorizer
#TF-IDF Vectorizer에 연결해주기
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)
tfidf_pos.vocabulary_

len(tfidf_pos.vocabulary_)

# 실제 문장에 단어를 토큰값으로 변환해주는 작업 진행 : 8분정도 걸림
X_trains = tfidf_pos.transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trains, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier() # 하이퍼 파라미터 조절 X / 기본 모델

# 교차검증 - 예상 성능 확인해보기
from sklearn.model_selection import cross_val_score

rs = cross_val_score(rf_model, X_train, y_train, cv = 5)
print(rs)
print(rs.mean())

# 모델 학습 및 평가
rf_model.fit(X_train, y_train)
rf_model.score(X_train, y_train)
rf_model.score(X_test, y_test)

# 하이퍼 파라미터 조합찾기
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 조절
# 내가 하이퍼 파라미터 뭉텡이를 던져주면 조합식을 짜서 최적의 조합을 짜줌
# 이때 사용하는건 교차검증 점수
param = { 'n_estimators' : [500, 1000, 1500, 2000, 2500], # 트리의 갯수 갯수가 많을수록 시간이 오래걸림
          'max_depth': range(1,6), # 트리의 최대 깊이 최대 깊이는 5를 안넘는게 좋음...
          #'min_samples_split': range(1,6),
          #'min_samples_leaf' : range(1,6)

        }

# range(100, 5001, 100) - 100부터 5000까지 100씩 증가 시켜주세요
grid_model = GridSearchCV(rf_model, param, n_jobs=-1, cv = 5)

grid_model.fit(X_train, y_train) #시간 좀 걸림

print(f" 최적 교차검증 점수 : {grid_model.best_score_}") # 최적 파라미터로 찾은 교차검증 점수
print(f" 최적 교차검증 파라미터 목록 : {grid_model.best_params_}") # 최적 파라미터 목록

best_model = grid_model.best_estimator_ # 최적 모델 만들기

best_model.fit(X_train, y_train) # 최적 모델 학습
best_model.score(X_test, y_test) # 최적 모델 평가 점수

# rf 활용
# predict_proba : 각 클래스별 불확실성 확률 -> 예측확률을 정답별로 표시
# target_names = ['부정', '중립' ,'긍정']
target_names = np.array(["부정", "중립", "긍정"])
review = ['이 화장품은 촉촉해서 건성 피부를 가지신 분이라면 좋을것 같아요']
vect_review = tfidf_pos.transform(review) # 토큰화 변환 진행 코드
pre = rf_model.predict(vect_review)
print(f"{review[0]} 문장은 {rf_model.predict_proba(vect_review).max()*100:.2f}%로 {target_names[pre[0]]} 리뷰입니다!")

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# 데이터 로드
data = pd.read_csv('./data/review_before.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.columns = ['제품명', '닉네임', '나이', '피부타입', '성별', '평점', '리뷰내용']

# 평점 기준
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if 'Verb' in d.index or 'Noun' in d.index or 'Adjective' in d.index:
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)

# 리뷰 내용 벡터화
X_trains = tfidf_pos.transform(X)

# 모델 로드 및 학습
# 실제로는 학습된 모델을 로드해야 합니다. 여기서는 기본 모델을 사용합니다.
rf_model = RandomForestClassifier()  # 하이퍼파라미터 조정된 모델로 교체 필요
rf_model.fit(X_trains, y)

# 예측 수행
probs = rf_model.predict_proba(X_trains)

# 최대 확률 및 예측 클래스 계산
max_probs = probs.max(axis=1) * 100
target_names = np.array(["부정", "중립", "긍정"])
predicted_classes = target_names[np.argmax(probs, axis=1)]

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame({
    '리뷰내용': X,
    '부정 확률 (%)': probs[:, 0] * 100,
    '중립 확률 (%)': probs[:, 1] * 100,
    '긍정 확률 (%)': probs[:, 2] * 100,
    '예측된 클래스': predicted_classes
})

# CSV 파일로 저장
results_df.to_csv('./data/review_predictions.csv', index=False)

print("리뷰 분석 결과를 'review_predictions.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 'review_predictions.csv'로 저장했습니다.


In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# 데이터 로드
data = pd.read_csv('./data/review_before.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.columns = ['제품명', '닉네임', '나이', '피부타입', '성별', '평점', '리뷰내용']

# 평점 기준
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']
product_names = data['제품명']  # 제품명 컬럼 추가

# 텍스트 감성 레이블 변환 함수
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if 'Verb' in d.index or 'Noun' in d.index or 'Adjective' in d.index:
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)

# 리뷰 내용 벡터화
X_trains = tfidf_pos.transform(X)

# 모델 로드 및 학습
rf_model = RandomForestClassifier()  # 하이퍼파라미터 조정된 모델로 교체 필요
rf_model.fit(X_trains, y)

# 예측 수행
probs = rf_model.predict_proba(X_trains)

# 최대 확률 및 예측 클래스 계산
max_probs = probs.max(axis=1) * 100
target_names = np.array(["부정", "중립", "긍정"])
predicted_classes = target_names[np.argmax(probs, axis=1)]

# 실제 평점을 텍스트로 변환
actual_sentiments = y.apply(get_sentiment_label)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame({
    '제품명': product_names,
    '리뷰내용': X,
    '실제 평점': actual_sentiments,
    '부정 확률 (%)': probs[:, 0] * 100,
    '중립 확률 (%)': probs[:, 1] * 100,
    '긍정 확률 (%)': probs[:, 2] * 100,
    '예측된 클래스': predicted_classes
})

# CSV 파일로 저장
results_df.to_csv('./data/review_predictions_with_product.csv', index=False)

print("리뷰 분석 결과를 'review_predictions_with_product.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 'review_predictions_with_product.csv'로 저장했습니다.


In [3]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# 데이터 로드
data = pd.read_csv('./data/review_before.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.columns = ['제품명', '닉네임', '나이', '피부타입', '성별', '평점', '리뷰내용']

# 평점 기준
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']
product_names = data['제품명']  # 제품명 컬럼 추가

# 텍스트 감성 레이블 변환 함수
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if 'Verb' in d.index or 'Noun' in d.index or 'Adjective' in d.index:
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)

# 리뷰 내용 벡터화
X_trains = tfidf_pos.transform(X)

# 모델 로드 및 학습
rf_model = RandomForestClassifier()  # 하이퍼파라미터 조정된 모델로 교체 필요
rf_model.fit(X_trains, y)

# 예측 수행
probs = rf_model.predict_proba(X_trains)

# 긍정, 부정, 중립 점수 계산
positive_scores = 2 * probs[:, 2] - 1
negative_scores = 1 - 2 * probs[:, 0]
neutral_scores = np.zeros_like(probs[:, 1])  # 중립 점수는 0으로 설정

# 최종 점수 계산 (긍정 점수와 부정 점수를 합산하고 중립 점수는 0으로)
final_scores = positive_scores + negative_scores + neutral_scores

# 실제 평점을 텍스트로 변환
actual_sentiments = y.apply(get_sentiment_label)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame({
    '제품명': product_names,
    '리뷰내용': X,
    '실제 평점': actual_sentiments,
    '긍정 확률 (%)': probs[:, 2] * 100,
    '부정 확률 (%)': probs[:, 0] * 100,
    '중립 확률 (%)': probs[:, 1] * 100,
    '점수': final_scores
})

# CSV 파일로 저장
results_df.to_csv('./data/review_predictions_with_final_scores.csv', index=False)

print("리뷰 분석 결과를 'review_predictions_with_final_scores.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 'review_predictions_with_final_scores.csv'로 저장했습니다.


In [3]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# 데이터 로드
data = pd.read_csv('./data/result_review.csv')
# data.drop('Unnamed: 0', axis=1, inplace=True)
data.columns = ['리뷰 번호', '제품명', '닉네임','나이','피부 타입','성별', '평점', '리뷰내용', '나이 그룹']

# 평점 기준
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']
product_names = data['제품명']  # 제품명 컬럼 추가

# 텍스트 감성 레이블 변환 함수
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if 'Verb' in d.index or 'Noun' in d.index or 'Adjective' in d.index:
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)

# 리뷰 내용 벡터화
X_trains = tfidf_pos.transform(X)

# 모델 로드 및 학습
rf_model = RandomForestClassifier()  # 하이퍼파라미터 조정된 모델로 교체 필요
rf_model.fit(X_trains, y)

# 예측 수행
probs = rf_model.predict_proba(X_trains)

# 긍정, 부정, 중립 점수 계산
positive_scores = 2 * probs[:, 2] - 1
negative_scores = 1 - 2 * probs[:, 0]
neutral_scores = np.zeros_like(probs[:, 1])  # 중립 점수는 0으로 설정

# 최종 점수 계산 (긍정 점수와 부정 점수를 합산하고 중립 점수는 0으로)
final_scores = positive_scores + negative_scores + neutral_scores

# 실제 평점을 텍스트로 변환
actual_sentiments = y.apply(get_sentiment_label)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame({
    '제품명': product_names,
    '리뷰내용': X,
    '실제 평점': actual_sentiments,
    '긍정 확률 (%)': probs[:, 2] * 100,
    '부정 확률 (%)': probs[:, 0] * 100,
    '중립 확률 (%)': probs[:, 1] * 100,
    '점수': final_scores
})

# CSV 파일로 저장
results_df.to_csv('./data/review6700_predictions_with_final_scores.csv', index=False)

print("리뷰 분석 결과를 'review6700_predictions_with_final_scores.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 'review6700_predictions_with_final_scores.csv'로 저장했습니다.


In [2]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# 데이터 로드 및 컬럼 이름 설정
data = pd.read_csv('./data/result_review_all_18.csv')
data.columns = ['리뷰 번호', '제품명', '닉네임', '나이', '피부 타입', '성별', '평점', '리뷰내용', '나이 그룹']

# 평점 기준 감성 분석 레이블 설정
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if ('Verb' in d.index) | ('Noun' in d.index) | ('Adjective' in d.index):
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)
X_trains = tfidf_pos.transform(X)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_trains, y, test_size=0.2, random_state=42)

# 랜덤포레스트 모델 및 교차검증
rf_model = RandomForestClassifier()
grid_model = GridSearchCV(rf_model, param_grid={
    'n_estimators': [500, 1000, 1500, 2000, 2500],
    'max_depth': range(1, 6)
}, n_jobs=-1, cv=5)
grid_model.fit(X_train, y_train)

# 최적 모델 추출 및 학습
best_model = grid_model.best_estimator_
best_model.fit(X_train, y_train)

# 예측 수행
probs = best_model.predict_proba(X_trains)

# 긍정, 부정, 중립 점수 계산
positive_scores = 2 * probs[:, 2] - 1
negative_scores = 1 - 2 * probs[:, 0]
neutral_scores = np.zeros_like(probs[:, 1])  # 중립 점수는 0으로 설정

# 최종 점수 계산
final_scores = positive_scores + negative_scores + neutral_scores

# 텍스트로 변환된 실제 평점
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

actual_sentiments = y.apply(get_sentiment_label)

# 기존 데이터와 예측 결과 결합
results_df = data.copy()  # 기존 데이터 복사
results_df['실제 평점'] = actual_sentiments
results_df['긍정 확률 (%)'] = probs[:, 2] * 100
results_df['부정 확률 (%)'] = probs[:, 0] * 100
results_df['중립 확률 (%)'] = probs[:, 1] * 100
results_df['점수'] = final_scores

# CSV 파일로 저장
results_df.to_csv('./data/result_review_all_18_final_scores.csv', index=False)

print("리뷰 분석 결과를 'result_review_all_18_final_scores.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 'result_review_all_18_final_scores.csv'로 저장했습니다.


In [7]:
positive_scores = 2 * 1 - 1
negative_scores = 1 - 2 * 0

# 최종 점수 계산
final_scores = positive_scores + negative_scores 
final_scores

2

In [4]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/124.9 MB 1.7 MB/s eta 0:01:16
   ---------------------------------------- 0.9/124.9 MB 11.4 MB/s eta 0:00:11
   ---------------------------------------- 1.2/124.9 MB 12.3 MB/s eta 0:00:11
   ---------------------------------------- 1.3/124.9 MB 7.4 MB/s eta 0:00:17
   ---------------------------------------- 1.4/124.9 MB 6.4 MB/s eta 0:00:20
    --------------------------------------- 1.6/124.9 MB 6.2 MB/s eta 0:00:21
    --------------------------------------- 2.4/124.9 MB 7.5 MB/s eta 0:00:17
   -- ------------------------------------- 6.4/124.9 MB 17.8 MB/s eta 0:00:07
   -- ------------------------------------- 7.5/124.9 MB 18.5 MB/s eta 0:00:07
   -- ------------------------------------- 7.8/124.9 MB 17.3 MB/s eta 0:00:07
   -- ------------------------------------- 8.3/124.9 MB 16.5 MB/s eta 0:00:08
   --- ------------------------------------ 12.0/124.9 MB 38.5 MB

In [5]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb  # XGBoost 라이브러리 임포트

# 데이터 로드 및 컬럼 이름 설정
data = pd.read_csv('./data/result_review_18.csv')
data.columns = ['리뷰 번호', '제품명', '닉네임', '평점', '리뷰내용']


# 평점 기준 감성 분석 레이블 설정
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if ('Verb' in d.index) | ('Noun' in d.index) | ('Adjective' in d.index):
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)
X_trains = tfidf_pos.transform(X)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_trains, y, test_size=0.2, random_state=42)

# XGBoost 모델 및 교차검증
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist', use_label_encoder=False)  # GPU 사용 설정
grid_model = GridSearchCV(xgb_model, param_grid={
    'n_estimators': [500, 1000, 1500, 2000, 2500],
    'max_depth': range(1, 6),
    'learning_rate': [0.01, 0.1, 0.2]
}, n_jobs=-1, cv=5)
grid_model.fit(X_train, y_train)

# 최적 모델 추출 및 학습
best_model = grid_model.best_estimator_
best_model.fit(X_train, y_train)

# 예측 수행
probs = best_model.predict_proba(X_trains)

# 긍정, 부정, 중립 점수 계산
positive_scores = 2 * probs[:, 2] - 1
negative_scores = 1 - 2 * probs[:, 0]
neutral_scores = np.zeros_like(probs[:, 1])  # 중립 점수는 0으로 설정

# 최종 점수 계산
final_scores = positive_scores + negative_scores + neutral_scores

# 텍스트로 변환된 실제 평점
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

actual_sentiments = y.apply(get_sentiment_label)

# 기존 데이터와 예측 결과 결합
results_df = data.copy()  # 기존 데이터 복사
results_df['실제 평점'] = actual_sentiments
results_df['긍정 확률 (%)'] = probs[:, 2] * 100
results_df['부정 확률 (%)'] = probs[:, 0] * 100
results_df['중립 확률 (%)'] = probs[:, 1] * 100
results_df['점수'] = final_scores

# CSV 파일로 저장
results_df.to_csv('./data/result_review_18_final_scores.csv', index=False)

print("리뷰 분석 결과를 'result_review_18_final_scores.csv'로 저장했습니다.")

C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: 
All the 375 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1531, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\smhrd\anaconda3\Lib\site-package

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# 데이터 로드 및 전처리
data = pd.read_csv('./data/review_before.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.columns = ['제품명', '닉네임', '나이', '피부타입', '성별', '평점', '리뷰내용']

# 평점 기준 감성 분석 레이블 설정
def rating_replace(rating):
    if rating in [5, 4]:  # 긍정은 2로 변경
        return 2
    elif rating == 3:  # 중립은 1로 변경
        return 1
    else:  # 부정은 0으로 변경
        return 0

data['감성분석 결과'] = data['평점'].apply(rating_replace)

# 리뷰 내용 및 감성분석 결과
X = data['리뷰내용']
y = data['감성분석 결과']

# 토큰화 함수 정의
okt = Okt()

def myTokenizer(text):
    d = pd.DataFrame(okt.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)
    if ('Verb' in d.index) | ('Noun' in d.index) | ('Adjective' in d.index):
        return d.loc[d.index.intersection(['Verb', 'Noun', 'Adjective']), 'morph'].values
    else:
        return []

# TF-IDF 벡터화
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer)
tfidf_pos.fit(X)
X_trains = tfidf_pos.transform(X)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_trains, y, test_size=0.2, random_state=42)

# 랜덤포레스트 모델 및 교차검증
rf_model = RandomForestClassifier()
grid_model = GridSearchCV(rf_model, param_grid={
    'n_estimators': [500, 1000, 1500, 2000, 2500],
    'max_depth': range(1, 6)
}, n_jobs=-1, cv=5)
grid_model.fit(X_train, y_train)

# 최적 모델 추출 및 학습
best_model = grid_model.best_estimator_
best_model.fit(X_train, y_train)

# 예측 수행
probs = best_model.predict_proba(X_trains)

# 긍정, 부정, 중립 점수 계산
positive_scores = 2 * probs[:, 2] - 1
negative_scores = 1 - 2 * probs[:, 0]
neutral_scores = np.zeros_like(probs[:, 1])  # 중립 점수는 0으로 설정

# 최종 점수 계산
final_scores = positive_scores + negative_scores + neutral_scores

# 텍스트로 변환된 실제 평점
def get_sentiment_label(value):
    if value == 2:
        return '긍정'
    elif value == 1:
        return '중립'
    else:
        return '부정'

actual_sentiments = y.apply(get_sentiment_label)

# 기존 데이터와 예측 결과 결합
results_df = data.copy()  # 기존 데이터 복사
results_df['실제 평점'] = actual_sentiments
results_df['긍정 확률 (%)'] = probs[:, 2] * 100
results_df['부정 확률 (%)'] = probs[:, 0] * 100
results_df['중립 확률 (%)'] = probs[:, 1] * 100
results_df['점수'] = final_scores

# CSV 파일로 저장
results_df.to_csv('./data/0902_test.csv', index=False)

print("리뷰 분석 결과를 '0902_test.csv'로 저장했습니다.")


C:\Users\smhrd\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


리뷰 분석 결과를 '0902_test.csv'로 저장했습니다.
